# 0. Introducción

<!-- <img src="https://raw.githubusercontent.com/JoePortilla/EnergyProductionCol-EDA/main/img/energyproduction_eda_cover.webp?token=GHSAT0AAAAAAB6AMADC5TFIULPHAY4XD3FUZBR77PA" width="800" alt="logo" style="display:block;margin:auto;"> -->



## Objetivo general
Work in progress.

## Objetivo especifico
Este notebook se centra en explorar y visualizar el conjunto de datos.

## Paquetes utilizados
- Procesamiento de datos
    - Pandas.
- Procesamiento numérico.
    - NumPy
- Visualización de datos
    - Seaborn
    - Matplotlib.

## Dataset

## Referencias

## Instructor
Joseph Santiago Portilla - Ingeniero Electrónico.
- GitHub: https://github.com/JoePortilla
- Twitter: https://twitter.com/JoePortilla
- LinkedIn: https://www.linkedin.com/in/PortillaJoe
- Kaggle: https://www.kaggle.com/joeportilla


# 1. Ajustes Iniciales

## 1.1 Importación de Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 1.2. Definición de estilo de gráficas

In [2]:
# https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
plt.style.use("seaborn-whitegrid")

# 2. Carga de Dataset y Revisión inicial

## 2.1. Lectura de CSV

In [3]:
df0 = pd.read_csv('/kaggle/input/all-space-missions-from-1957/Space_Corrected.csv')

## 2.2. Exploración inicial del dataframe

### Info. general del dataset

El conjunto de datos contiene las siguientes variables:

In [4]:
df0.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Company Name', 'Location', 'Datum',
       'Detail', 'Status Rocket', ' Rocket', 'Status Mission'],
      dtype='object')

In [5]:
df0.head()

,Unnamed: 0,Unnamed: 0.1,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [6]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      4324 non-null   int64 
 1   Unnamed: 0.1    4324 non-null   int64 
 2   Company Name    4324 non-null   object
 3   Location        4324 non-null   object
 4   Datum           4324 non-null   object
 5   Detail          4324 non-null   object
 6   Status Rocket   4324 non-null   object
 7    Rocket         964 non-null    object
 8   Status Mission  4324 non-null   object
dtypes: int64(2), object(7)
memory usage: 304.2+ KB


### Dimensión del datafrane

In [7]:
df0.shape

(4324, 9)

# 3. Preprocesamiento

In [8]:
df = df0.copy()

## 3.1. Feature selection

In [9]:
filt = ['Unnamed: 0', 'Unnamed: 0.1']
df.drop(filt, axis=1, inplace=True)

In [10]:
df.head()

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


## 3.2. Column renaming

In [11]:
df.rename(columns={'Company Name':'company',
                   'Location':'launch_facility',
                   'Datum':'date',
                   'Detail':'detail',
                   'Status Rocket':'rocket_status',
                   ' Rocket':'cost',
                   'Status Mission':'mission_status'},
          inplace=True)

## 3.3. Data filtering

-

## 3.4. Feature conversion

### 3.4.1. Fecha

Eliminar los caracteres ' UTC' de los registros de fecha

In [12]:
df['date'] = df['date'].str.replace(' UTC', '')

Debido a que hay formatos combinados en los registros de fecha se crea una función para poder convertirlos a tipo `datetime`

In [13]:
def date_formator(date):
    if ':' in date:
        return pd.to_datetime(date, format='%a %b %d, %Y %H:%M')
    else:
        return pd.to_datetime(date, format='%a %b %d, %Y')

Convertir columna `date` a tipo datetime

In [14]:
df['date'] = df['date'].apply(date_formator)

## 3.5. Feature construction

### 3.5.1. Año y mes

Crear columna de año y mes

In [15]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

### 3.5.2. Ubicación lanzamiento

Crear columna con la ubicación del lanzamiento, a partir del centro de lanzamiento

In [16]:
df['launch_location'] = df['launch_facility'].str.split(', ').str[-1]

### 3.5.3. Cohete y payload

Dividir la columna `detail` en dos. Una para el nombre del cohete `rocket` y otra para la carga `payload` que lleva.

In [17]:
df[['rocket', 'payload']] = df['detail'].str.split('|', expand=True)
df.drop('detail', axis=1, inplace=True)

## 3.6. Value renaming

-

## 3.7. Data reordering

In [18]:
df = df.sort_values('date')
df = df.reset_index(drop=True)

## 3.8. Dataframe preprocesado

In [19]:
df.head(10)

,company,launch_facility,date,rocket_status,cost,mission_status,year,month,launch_location,rocket,payload
0,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-10-04 19:28:00,StatusRetired,NaN,Success,1957,10,Kazakhstan,Sputnik 8K71PS,Sputnik-1
1,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03 02:30:00,StatusRetired,NaN,Success,1957,11,Kazakhstan,Sputnik 8K71PS,Sputnik-2
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06 16:44:00,StatusRetired,NaN,Failure,1957,12,USA,Vanguard,Vanguard TV3
3,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01 03:48:00,StatusRetired,NaN,Success,1958,2,USA,Juno I,Explorer 1
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05 07:33:00,StatusRetired,NaN,Failure,1958,2,USA,Vanguard,Vanguard TV3BU
5,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-03-05 18:27:00,StatusRetired,NaN,Failure,1958,3,USA,Juno I,Explorer 2
6,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-03-17 12:15:00,StatusRetired,NaN,Success,1958,3,USA,Vanguard,Vanguard 1
7,AMBA,"LC-5, Cape Canaveral AFS, Florida, USA",1958-03-26 17:38:00,StatusRetired,NaN,Success,1958,3,USA,Juno I,Explorer 3
8,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1958-04-27 09:01:00,StatusRetired,NaN,Failure,1958,4,Kazakhstan,Sputnik 8A91,Sputnik-3 #1
9,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-04-28 02:53:00,StatusRetired,NaN,Failure,1958,4,USA,Vanguard,Vanguard TV5


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   company          4324 non-null   object        
 1   launch_facility  4324 non-null   object        
 2   date             4324 non-null   datetime64[ns]
 3   rocket_status    4324 non-null   object        
 4   cost             964 non-null    object        
 5   mission_status   4324 non-null   object        
 6   year             4324 non-null   int64         
 7   month            4324 non-null   int64         
 8   launch_location  4324 non-null   object        
 9   rocket           4324 non-null   object        
 10  payload          4324 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 371.7+ KB


# X. Conclusiones EDA

- Work in progress
-  